# `Step: 0: Config`
- 1-) Activate compherensive config file 

In [ ]:
try: 
    if manager == 1: 
        print("Manager Mode: ON")
except:
    print("Manager Mode: OFF")
    %run s0_config.ipynb 
    import scanpy as sc 

# `Step: 1: Load`
- 1-) Read Data  

## 1-) `Read` 

#### Operation 

In [ ]:
access_data_path("process_0/data", "blood_atlas_of_COVID.h5ad")

In [ ]:
point_start()

#%memit adata = sc.read_h5ad(import_data_path, chunk_size=1000) 
%memit adata = sc.read(access_data_path("process_0/data", "blood_atlas_of_COVID.h5ad")) 
adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx` 

point_end() 

In [ ]:
import numpy as np

np.random.seed(0)

def downsample(group, n_samples=10000):
    if len(group) > n_samples:
        return group.sample(n_samples)
    return group

adata = adata[adata.obs['cell_type'].groupby(adata.obs['cell_type']).apply(
    lambda x: downsample(x)).index.get_level_values(1)] 

In [ ]:
#adata.write(check_point_s1) 

## 2-) `Pipeline Management`

In [ ]:
# `Step 2: Preprocessing`
pipeline_Highly_Expressed_Genes = True 
pipeline_Basic_Filtering        = True 
pipeline_MT_Metrics             = True 
pipeline_doublet_detection      = True
pipeline_Outlier                = True 
pipeline_Normalization          = True 
pipeline_Log_Transformation     = True 
pipeline_Highly_variable_genes  = True 
pipeline_Regress_Out            = True 
pipeline_Scaling                = True 

## 3-) `Overview`

In [ ]:
adata.raw

In [ ]:
adata 

In [ ]:
adata.obs.T

---
## 4-) `EDA`

In [ ]:
"""
try: 
    if manager == 1: 
        print("Manager Mode: ON")
except:
    print("Manager Mode: OFF")
    %run s0_config.ipynb 

adata = sc.read(check_point_s1)    
"""

In [ ]:
adata 

In [ ]:
adata.obs["cell_type"].value_counts() 

In [ ]:
print("Number of unique cell types: ", len(adata.obs["cell_type"].value_counts())) 

In [ ]:
adata.obs["disease"].value_counts() 

In [ ]:
adata.obs["tissue"].value_counts() 

In [ ]:
adata.obs["organism"].value_counts() 

In [ ]:
adata.obs["sex"].value_counts() 

In [ ]:
adata.obs["development_stage"].value_counts() 

In [ ]:
subject_disease_status     = "mixed"
subject_tissue_status      = "blood"
subject_organism_status    = "Homo sapiens"
subject_sex_status         = "mixed"
subject_development_status = "7-year-old human stage"

In [ ]:
adata.obs["disease"].value_counts() 

In [ ]:
#adata = adata[adata.obs['disease'] == subject_disease_status]
print(len(adata))  

In [ ]:
adata.obs["tissue"].value_counts() 

In [ ]:
#adata = adata[adata.obs['tissue'] == subject_tissue_status]

print(len(adata))  

In [ ]:
adata.obs["organism"].value_counts() 

In [ ]:
#adata = adata[adata.obs['organism'] == subject_organism_status]
print(len(adata))  

In [ ]:
adata.obs["sex"].value_counts() 

In [ ]:
#adata = adata[adata.obs['sex'] == subject_sex_status]
print(len(adata))  

In [ ]:
adata.obs["development_stage"].value_counts() 

In [ ]:
#adata = adata[adata.obs['development_stage'] != "1-year-old human stage"].copy() 
#adata = adata[adata.obs['development_stage'] != "2-year-old human stage"].copy() 
#adata = adata[adata.obs['development_stage'] != "3-year-old human stage"].copy() 
#adata = adata[adata.obs['development_stage'] != "4-year-old human stage"].copy() 
#adata = adata[adata.obs['development_stage'] != "5-year-old human stage"].copy() 
#adata = adata[adata.obs['development_stage'] != "6-year-old human stage"].copy() 
#adata = adata[(adata.obs['development_stage'] == "6-year-old human stage") | (adata.obs['development_stage'] == "7-year-old human stage")].copy() 
adata = adata[adata.obs['development_stage'] == "7-year-old human stage"].copy() 
#adata = adata[adata.obs['development_stage'] != "8-year-old human stage"].copy() 
#adata = adata[adata.obs['development_stage'] != "9-year-old human stage"].copy() 
print(len(adata))  

In [ ]:
#adata = adata[adata.obs['disease'] == "normal"].copy()   

In [ ]:
adata

In [ ]:
adata.obs['cell_type'].value_counts() 

----
# Fixing the Gene Marker Labelling

In [ ]:
print(adata.var_names[:10],adata.var_names[100:110],adata.var_names[10:]) 

In [ ]:
# Load the data
true_data = access_data_path("process_0/data", "rna_single_cell_type.tsv")
ensembl_to_symbol = pd.read_csv(true_data, sep='\t')

# Process the mapping file
unique_ensembl_to_symbol = ensembl_to_symbol[['Gene', 'Gene name']].drop_duplicates('Gene')
unique_ensembl_to_symbol['Gene'] = unique_ensembl_to_symbol['Gene'].str.strip()
unique_ensembl_to_symbol['Gene name'] = unique_ensembl_to_symbol['Gene name'].str.strip()
unique_ensembl_to_symbol.set_index('Gene', inplace=True)
unique_ensembl_to_symbol_DF = unique_ensembl_to_symbol.reset_index()
unique_ensembl_to_symbol_DF.columns = ["ensembl_id", "gene_symbol"]

# Create the mapping dictionary
ensembl_to_symbol_map = unique_ensembl_to_symbol_DF.set_index('ensembl_id')['gene_symbol'].to_dict()


In [ ]:
ensembl_to_symbol = pd.read_csv(true_data, sep='\t')
ensembl_to_symbol 

In [ ]:
filtered_df = ensembl_to_symbol[~ensembl_to_symbol['Gene name'].str.startswith('ENSG')] 
len(pd.unique(filtered_df["Gene name"]))

In [ ]:

# Map Ensembl IDs to gene symbols
gene_symbols = adata.var_names.map(ensembl_to_symbol_map)
gene_symbols = pd.Series(gene_symbols, index=adata.var_names).fillna(pd.Series(adata.var_names, index=adata.var_names))

# Update adata with the new gene symbols
adata.var['gene_symbols'] = gene_symbols

# Ensure there are no conflicts between the index and the new gene_symbols column
if 'gene_symbols' in adata.var.columns:
    adata.var.reset_index(inplace=True)  # Temporarily reset index to allow updating var_names
    adata.var.set_index('gene_symbols', inplace=True)
else:
    adata.var_names = adata.var['gene_symbols']
    adata.var_names_make_unique()

In [ ]:
adata.var_names_make_unique() 

In [ ]:
print(np.array(gene_symbols)[:30] )

In [ ]:

# Check the updated var_names
print(adata.var_names[:10],adata.var_names[100:110],adata.var_names[10:])  

## End
---

In [ ]:
true_label_column = "cell_type" 

In [ ]:
adata.var

---
# `Step 2: Preprocessing`
- 1-) Highly Expressed Genes 
- 2-) Basic Filtering 
- 3-) MT Metrics
- 4-) Outlier 
- 5-) Normalization  
- 6-) Log Transformation  
- 7-) Highly Variable Genes
- 8-) Regress Out 
- 9-) Scaling 

## 1-) `Highly Expressed Genes`

##### Result: | Scatter (pl)

In [ ]:
if pipeline_Highly_Expressed_Genes == True:
    %memit sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
adata.obs['cell_type'].value_counts() 

## 2-) `Basic Filtering` 

##### Operation: Calculate (pp)

In [ ]:
adata

In [ ]:
if pipeline_Basic_Filtering == True: 
    print("*" * 20)
    
    # Filter cells with minimum number of genes
    print(f"Before: Minimum Genes: {adata.n_obs} cells, {adata.n_vars} genes") 
    sc.pp.filter_cells(adata, min_genes=200)
    print(f"After: Minimum Genes: {adata.n_obs} cells, {adata.n_vars} genes") 
    print("*" * 20)
    
    # Filter cells with maximum number of genes
    print(f"Before: Maximum Genes: {adata.n_obs} cells, {adata.n_vars} genes") 
    sc.pp.filter_cells(adata, max_genes=2500)
    print(f"After: Maximum Genes: {adata.n_obs} cells, {adata.n_vars} genes") 
    print("*" * 20)
    
    # Filter cells with minimum number of counts
    print(f"Before: Minimum Counts: {adata.n_obs} cells, {adata.n_vars} genes") 
    sc.pp.filter_cells(adata, min_counts=500)
    print(f"After: Minimum Counts: {adata.n_obs} cells, {adata.n_vars} genes") 
    print("*" * 20)
    
    # Filter cells with maximum number of counts
    print(f"Before: Maximum Counts: {adata.n_obs} cells, {adata.n_vars} genes") 
    sc.pp.filter_cells(adata, max_counts=25000)
    print(f"After: Maximum Counts: {adata.n_obs} cells, {adata.n_vars} genes") 
    print("*" * 20)
    
    # Filter genes expressed in fewer than a minimum number of cells
    print(f"Before: Minimum Cells per Gene: {adata.n_obs} cells, {adata.n_vars} genes")    
    sc.pp.filter_genes(adata, min_cells=3) 
    print(f"After: Minimum Cells per Gene: {adata.n_obs} cells, {adata.n_vars} genes")    
    print("*" * 20)
    
    # Filter genes expressed in more than a maximum number of cells
    print(f"Before: Maximum Cells per Gene: {adata.n_obs} cells, {adata.n_vars} genes")    
    sc.pp.filter_genes(adata, max_cells=adata.n_obs)
    print(f"After: Maximum Cells per Gene: {adata.n_obs} cells, {adata.n_vars} genes")    
    print("*" * 20)


In [ ]:
adata

In [ ]:
adata.obs['cell_type'].value_counts() 

## 3-) `MT Metrics`

##### Operation: Calculate (pp)

In [ ]:
print(adata.var_names[:50])

In [ ]:
# (1/2)
%memit adata.var["mt"] = adata.var_names.str.startswith("MT-")
# (2/2)

if pipeline_MT_Metrics             == True:
    sc.pp.calculate_qc_metrics(
        adata, qc_vars  = ["mt"], 
        percent_top     = None, 
        log1p           = False, 
        inplace         = True
    )  


In [ ]:
adata.var["mt"] 

In [ ]:
adata.obs['cell_type'].value_counts() 

##### Result: | Violin (pl)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter      = 0.4,
    multi_panel = True,
) 

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter      = 0.1,
    multi_panel = True,
) 

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter      = 0.9,
    multi_panel = True,
) 

## 5-) `Outlier`

##### Result: | Scatter (pl)

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts") 

##### Operation 

In [ ]:
if pipeline_Outlier             == True:
    %memit adata = adata[adata.obs.n_genes_by_counts < 2500, :].copy()
    %memit adata = adata[adata.obs.total_counts < 3000, :].copy()
    %memit adata = adata[adata.obs.total_counts > 1000, :].copy()
    %memit adata = adata[adata.obs.pct_counts_mt < 4, :].copy()

In [ ]:
adata.obs['cell_type'].value_counts() 

##### Result: | Scatter (pl)

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts") 

## 4-) `Doublet Detection`

In [ ]:
# Step 4: Doublet Detection
if pipeline_doublet_detection:
    sc.pp.scrublet(adata, batch_key="scRNASeq_sample_ID") 


In [ ]:
pd.concat([adata.obs[['doublet_score', 'predicted_doublet']].head(10), adata.obs[['doublet_score', 'predicted_doublet']].tail(10)])  

In [ ]:
print("Detected Doublet:", adata.obs['predicted_doublet'].value_counts())
print("Detected Doublets:", adata[adata.obs['predicted_doublet'] == True] ) 

In [ ]:
# Filtering doublets
adata = adata[adata.obs['predicted_doublet'] == False]  

In [ ]:
adata 

## 5- `Normalization`  

##### Operation: Calculate (pp)

In [ ]:
if pipeline_Normalization         == True:
    %memit sc.pp.normalize_total(adata, target_sum=1e4)

## 6- `Log Transformation ` 

##### Operation: Calculate (pp)

In [ ]:
if pipeline_Log_Transformation         == True:
    %memit sc.pp.log1p(adata) 

In [ ]:
adata.obs['cell_type'].value_counts() 

## 7-) `Highly Variable Genes`

##### Operation: Calculate (pp)

In [ ]:
if pipeline_Highly_variable_genes         == True:
    %memit sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
adata.obs['cell_type'].value_counts() 

##### Result (pl) | Scatter 

In [ ]:
%memit sc.pl.highly_variable_genes(adata) # Now you can plot the highly variable genes 

##### Operation: Save Raw 

In [ ]:
adata.raw = adata

##### Operation: Apply 

In [ ]:
adata = adata[:, adata.var.highly_variable]

## 8-) `Regress Out `

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

##### Operation: Regress Out (pp)

In [ ]:
if pipeline_Regress_Out == True:
    %memit sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"]) 

In [ ]:
adata.obs['cell_type'].value_counts() 

## 9-) `Scaling `

##### Result 

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

##### Operation: Calculate (pp)

In [ ]:
if pipeline_Scaling == True:
    sc.pp.scale(adata, max_value=10)  

In [ ]:
adata.obs['cell_type'].value_counts() 

##### Result 

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

# `Step 2: Checkpoint` 

#### Operation: Checkpoint (Save) 
Save the result.

In [ ]:
adata.write(check_point_s2)

In [ ]:
adata

In [ ]:
#stop 

---
# `Step 3: Dimension Reduction | Embedding`
- 1-) PCA 
- 2-) Detection of Neighbours
- 3-) UMAP 
- 4-) T-SNE

## 1-) `PCA` 

#### Operation: Calculate (tl)

In [ ]:
%memit sc.tl.pca(adata, svd_solver='arpack') 
#%memit sc.tl.pca(adata, svd_solver='randomized') 

In [ ]:
#sc.pl.pca(adata, color="CST3")

In [ ]:
adata.obs['cell_type'].value_counts() 

#### Result & Operation (pl)
- TO DO: Select Parameters
- Parameter 1: n_neighbors
- Parameter 2: n_pcs 

In [ ]:
%memit sc.pl.pca_variance_ratio(adata, log=True)

## 2-) `Detection of Neighbours`

#### Operation: Calculate (pp)

In [ ]:
"""from joblib import parallel_backend
with parallel_backend('threading', n_jobs=16):
        sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40) """

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=44)

In [ ]:
adata.obs['cell_type'].value_counts() 

## 3-) `UMAP`

#### Operation: Calculate (tl)

In [ ]:
sc.tl.umap(adata)

In [ ]:
adata.obs['cell_type'].value_counts() 

#### Results

In [ ]:
try:
    sc.pl.umap(adata, color=["ISG15", "FNDC10", "MT-CYB"], use_raw=True)
except: 
    pass 

In [ ]:
try:
    sc.pl.umap(adata, color=["ISG15", "FNDC10", "MT-CYB"], use_raw=False)
except: 
    pass 

## 4-) `T-SNE`

In [ ]:
sc.tl.tsne(adata)  

In [ ]:
adata.obs['cell_type'].value_counts() 

In [ ]:
try: 
    try: 
        sc.pl.tsne(adata, color=["ISG15", "FNDC10", "MT-CYB"], use_raw=False)  
    except:
        sc.pl.tsne(adata, color=["ISG15", "FNDC10", "MT-CYB"], use_raw=True)  
except:
    pass 

# `Step 3: Checkpoint` 

#### Operation: Checkpoint (Save) 
Save the result.

In [ ]:
try:
    adata.write(check_point_s3)
except Exception as e:
    print("Error",e)

In [ ]:
adata

---
# `Step 4: Clustering | Trajectory Inference`
- 1-) Leiden 
- 2-) Dendogram
- 3-) PAGA

## 1-) `Leiden `

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np


resolutions = np.arange(0.16, 1.0, 0.001)
cluster_counts = []
res_cluster_dict = {}

for res in resolutions:
    res_key = str(res)
    print("res_key:",res_key)
    sc.tl.leiden(
        adata,    
        key_added=res_key,  
        resolution=res,
        random_state=0,
        flavor="igraph",
        n_iterations=-1,  # Try with unlimited iterations
        directed=False,
    ) 
    # Retrieving the number of unique clusters
    res_cluster_dict[str(res)] = adata.obs[str(res)].nunique() 
    cluster_counts.append(adata.obs[str(res)].nunique()) 


In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(resolutions, cluster_counts, marker='o')
ax.set_xlabel('Leiden Resolution')
ax.set_ylabel('Number of Clusters')
ax.set_title('Cluster Count vs. Leiden Resolution')
ax.grid(True)
plt.show()
res_cluster_dict 

In [ ]:
adata.obs['cell_type_short'] = adata.obs['cell_type'].str[:8]

In [ ]:
# Plot UMAP for selected resolutions
optimal_resolutions = [res for res, count in res_cluster_dict.items() if count == 17]
print(f"Optimal Resolutions: {optimal_resolutions}")

for res in optimal_resolutions:
    sc.pl.umap(
        adata, 
        color=[str(res), 'cell_type_short'], 
        legend_loc="on data", 
        title=[f'Leiden Resolution: {res}', 'cell_type_short'], 
        frameon=True,
        size=10  # Adjust this value to control the dot size
    )

In [ ]:
print("Number of resolutions:", len(resolutions)) 
print("Number of unique cell types: ", len(adata.obs["cell_type"].value_counts()))  
print(res_cluster_dict) 

In [ ]:
best_resolution = 0.2670000000000001

## 1-) `Leiden ` (Selection)

In [ ]:
selected_obs_cluster_key   = f"leiden_{best_resolution}"
selected_leiden_resolution = best_resolution 
print(f"Selected Resolution: {selected_leiden_resolution}")
print(f"Selected Key: {selected_obs_cluster_key}")

#### Operation: Calculate (tl)

In [ ]:
current = 0
current_key         = selected_obs_cluster_key
current_resolution  = selected_leiden_resolution 

print(f"Key: {current_key}") 
print(f"Resolution: {current_resolution}")


sc.tl.leiden(
    adata,    
    key_added    = current_key, 
    resolution   = current_resolution,
    random_state = 0,
    flavor       = "igraph",
    n_iterations = 2,
    directed     = False,
) 
# Retrieve the number of unique clusters
current_Clusters = adata.obs[current_key].nunique() 

print(f"Number of clusters for {current_key}: {current_Clusters}") 
print("Number of True Clusters:", len(pd.unique(adata.obs["cell_type"])))
list(pd.unique(adata.obs["cell_type"]) ) 

In [ ]:
sc.pl.umap(
    adata, 
    color       = [current_key,  "cell_type_short", #'majority_voting_best_match', 'majority_voting_prob_match',
                   ], 
    legend_loc  = "on data", 
    title       = [current_key,  "cell_type_short",#'majority_voting_best_match', 'majority_voting_prob_match',
                   ], 
    frameon     = True, 
    #save=".pdf"
)
print(f"Number of CLusters inselected cluster key: {selected_obs_cluster_key} is :{adata.obs[selected_obs_cluster_key].nunique()}") 

## 2-) `PAGA `

#### Operation: Calculate (tl)

In [ ]:
sc.tl.paga(adata, groups = selected_obs_cluster_key)
sc.pl.paga(adata, plot=False)  # remove `plot=False` if one wants to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')

In [ ]:
adata.obs['cell_type'].value_counts() 

# `Step 4: Checkpoint` 

#### Operation: Checkpoint (Save) 
Save the result.

In [ ]:
try:
    adata.write(check_point_s4)
except Exception as e:
    print("Error:",e) 

In [ ]:
adata

---
# `Step 5: Ranked Genes`
- 1-) Leiden | t-test 
- 2-) Leiden | Wilcoxon
- 3-) Leiden | Logreg 

In [ ]:
# One can select other methods too. Wilcoxon is advised so we continue with it. 
switch_Leiden_ttest    = 0 
switch_Leiden_Wilcoxon = 1 
switch_Leiden_Logreg   = 0 

## 1-) `Leiden | t-test`

    Grouping: Uses the 'leiden' clustering results to define groups of cells.
    Method: Employs the Wilcoxon rank-sum test to identify differentially expressed genes between the groups.
    Default Settings: Other parameters are set to their default values, which means it compares all groups against the rest, does not correct for multiple testing explicitly (though p-values are adjusted), and does not filter genes based on variance or other criteria unless specified in adata.

In [ ]:
point_start()

if switch_Leiden_ttest == 1: 
    ranked_key_leiden_ttest = f'rank_genes_leiden_ttest'  

    #%memit sc.tl.rank_genes_groups(adata, groupby = 'leiden', method="t-test", key_added='rank_genes_leiden', reference='rest', n_genes=100)  # Adjust n_genes as needed 
    %memit sc.tl.rank_genes_groups(adata, groupby = selected_obs_cluster_key, method = "t-test", key_added = ranked_key_leiden_ttest, reference = 'rest', n_genes = 100)  # Adjust n_genes as needed 

    #sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)
    sc.pl.rank_genes_groups(adata, key = ranked_key_leiden_ttest, n_genes = 25, sharey = False)

point_end() 

In [ ]:
adata.obs['cell_type'].value_counts() 

In [ ]:
sc.settings.verbosity = 2  # reduce the verbosity

## 2-) `Leiden | Wilcoxon`

In [ ]:
point_start()

if switch_Leiden_Wilcoxon == 1: 

    ranked_key_leiden_wilcoxon = f'rank_genes_leiden_wilcoxon'  

    #%memit sc.tl.rank_genes_groups(adata, groupby = 'leiden', method="t-test", key_added='rank_genes_leiden', reference='rest', n_genes=100)  # Adjust n_genes as needed 
    %memit sc.tl.rank_genes_groups(adata, groupby = selected_obs_cluster_key, method = "wilcoxon", key_added = ranked_key_leiden_wilcoxon, reference = 'rest', n_genes = 100)  # Adjust n_genes as needed 

    #sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)
    sc.pl.rank_genes_groups(adata, key = ranked_key_leiden_wilcoxon, n_genes = 25, sharey = False)

    selected_uns_ranked_key = ranked_key_leiden_wilcoxon  # It is for future usage 

point_end() 

    

In [ ]:
adata.obs['cell_type'].value_counts() 

In [ ]:
sc.pl.rank_genes_groups(adata, key = ranked_key_leiden_wilcoxon, n_genes = 100, sharey = False) 

## 3-) `Leiden | Logreg`

In [ ]:
point_start()

if switch_Leiden_Logreg == 1: 

    ranked_key_leiden_logreg = f'rank_genes_leiden_logreg'  

    #%memit sc.tl.rank_genes_groups(adata, groupby = 'leiden', method="t-test", key_added='rank_genes_leiden', reference='rest', n_genes=100)  # Adjust n_genes as needed 
    %memit sc.tl.rank_genes_groups(adata, groupby = selected_obs_cluster_key, method = "logreg", key_added = ranked_key_leiden_logreg, reference = 'rest', n_genes = 100)  # Adjust n_genes as needed 

    #sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)
    sc.pl.rank_genes_groups(adata, key = ranked_key_leiden_logreg, n_genes = 25, sharey = False)

point_end() 


In [ ]:
adata.obs['cell_type'].value_counts() 

## 4-) `True Label | Wilcoxon`

In [ ]:
#temp_adata = adata.copy() 
#adata = temp_adata[temp_adata.obs["cell_type"] != "mast cell"]

In [ ]:
point_start()

sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=200, facecolor="white", figsize=(14, 14)) 

if switch_Leiden_Wilcoxon == 1: 

    true_label_obs_cluster_key = true_label_column
    true_label_uns_ranked_key  = f'rank_genes_true_label'  
    

    #%memit sc.tl.rank_genes_groups(adata, groupby = 'leiden', method="t-test", key_added='rank_genes_leiden', reference='rest', n_genes=100)  # Adjust n_genes as needed 
    %memit sc.tl.rank_genes_groups(adata, groupby = true_label_obs_cluster_key, method = "wilcoxon", key_added = true_label_uns_ranked_key, reference = 'rest', n_genes = 100)  # Adjust n_genes as needed 

    #sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)
    sc.pl.rank_genes_groups(adata, key = true_label_uns_ranked_key, n_genes = 100, sharey = False)


point_end() 

    

In [ ]:
adata.obs['cell_type'].value_counts() 

# `Step 5: Checkpoint` 

#### Operation: Checkpoint (Save) 
Save the result.

In [ ]:
print(f"Number of unique cell types in true labels: {len(pd.unique(adata.obs['cell_type']))}")
adata.obs['cell_type'].value_counts() 

In [ ]:
try:
    adata.write(check_point_s5)
except Exception as e:
    print("Error:",e) 

In [ ]:
adata

In [ ]:
target_gene_number = 25 

In [ ]:
df_1 = pd.DataFrame(adata.obs[selected_obs_cluster_key].value_counts())  
df_1 = df_1.reset_index(drop=False)
df_2 = pd.DataFrame(adata.obs["cell_type"].value_counts()  ) 
df_2 = df_2.reset_index(drop=False)
df_concat = pd.concat([df_1,df_2], axis=1) 
df_concat   

In [ ]:
# Set the verbosity and figure parameters
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=200, facecolor="white", figsize=(10, 10))

# Plot the UMAP with larger dots
sc.pl.umap(
    adata, 
    color=[selected_obs_cluster_key, true_label_obs_cluster_key], 
    legend_loc="on data", 
    title=[selected_obs_cluster_key, true_label_obs_cluster_key], 
    frameon=True,
    size=50  # Adjust this value to control the dot size
)

# Print the number of clusters
print(f"Number of Clusters in selected cluster key: {selected_obs_cluster_key} is :{adata.obs[selected_obs_cluster_key].nunique()}")


In [ ]:
Variables_Dict_s5 ={"target_gene_number"      : target_gene_number, 
                    
                    "selected_obs_cluster_key"           : selected_obs_cluster_key,
                    "selected_uns_ranked_key"            : selected_uns_ranked_key,
                    "selected_cluster_names"             : list(pd.unique(adata.obs[selected_obs_cluster_key])), 
                    "selected_sample_cluster_barcode"    : list(pd.unique(adata.obs[selected_obs_cluster_key]))[0], 
                    

                    "true_label_obs_cluster_key"           : true_label_obs_cluster_key,
                    "true_label_uns_ranked_key"            : true_label_uns_ranked_key,
                    "true_label_cluster_names"             : list(pd.unique(adata.obs[true_label_obs_cluster_key])), 
                    "true_label_sample_cluster_barcode"    : list(pd.unique(adata.obs[true_label_obs_cluster_key]))[0], 
                     }

export_name = check_point_s5_variables

with open( access_data_path(target_folder = f"process_0/checkpoint", target_file = export_name + ".pkl" )  , 'wb') as file:
    pickle.dump(Variables_Dict_s5, file)
